In [974]:
import re
import emoji
import random
import pandas as pd
from pandas import read_csv
from opencc import OpenCC
import os, itertools
from itertools import groupby

In [975]:
# load data
map_ = read_csv('Map_Ramen_data.csv', encoding='utf8',index_col=0)
fb_ = read_csv('store_id_done.csv', encoding='utf8',index_col=0)
fb_ = fb_.drop(columns=['store_id','post_id'])
#map_.head()

In [976]:
map_ = map_.sort_values(by=['store'])
map_ = map_.reset_index(drop=True)
map_ = map_.sort_index(axis=0 ,ascending=True)
map_ = map_.iloc[::-1]
map_ = map_.reset_index(drop=True)
#map_.head()

In [977]:
map_store = list(map_['store'])
map_store_lst = []

for store in map_store:
    store = store.lower()
    store = re.sub(r'[^\w\s]','',store)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬') \
                    .replace('鷹流東京豚骨拉麵極匠','鷹流極匠').replace('鷹流東京醬油拉麵蘭丸','鷹流蘭丸') \
                    .replace('#','').replace(' ','').replace(u'\u3000',u'').replace('柑橘shin','柑橘shinn') \
                    .replace('《','').replace('》','').replace('台','臺').replace('らーめん','拉麵').replace('の','之') \
                    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('ラーメン','拉麵') \
                    .replace('(','').replace(')','').replace('·','').replace('/','').replace('aqua2','').replace('麵鋪','麵舖') \
                    .replace('※','').replace('ぁ','あ').replace('²','2').replace('台湾','臺灣').replace('番茄','蕃茄') \
                    
    map_store_lst.append(store) 
                                                                              
#map_store_lst
map_['stem_map_store'] = map_store_lst
fb_store = list(fb_['stem_store'])
fb_store = sorted(list(set(fb_store)))
fb_store
all_store = map_store_lst + fb_store
shop_sorted = sorted(list(set(all_store)))
#shop_sorted

# Mapping store names (facebook and Google map)

In [978]:
### FIRST GROUPING
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))
#first_sort

In [979]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        #second_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[1])])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [980]:
#second_sort_len

In [981]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵':
        third_sort.append([item])
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
            #third_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[2])])
        elif len(item) > 1 and len(item[0]) <= 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [982]:
#third_sort_len

In [983]:
# Testing
lst = list(itertools.chain(*third_sort_len))
#print(len(lst)) 
#len(unique) 

In [984]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fourth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])
        
fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [985]:
### FIFTH GROUPING
fifth_sort = []

util_func = lambda x: x[4]
for item in fourth_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fifth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) > 7:
            temp = sorted(item, key = util_func)
            fifth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 4:
            fifth_sort.append([item])            
        elif len(item) > 1 and len(item[0]) >= 5 and len(item[0]) <= 7:
            fifth_sort.append([item])
        else:
            unique.append(item[0])
            

fifth_sort_len = []

for item_1 in fifth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fifth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [986]:
#fifth_sort_len

In [987]:
### CHECK UNIQUE STORE NAME
#unique

In [988]:
done = []
unclassified_group = []

for item in fifth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    for i in range(len(item)-1):
        if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
            grouped.append(item[i+1])
        else:
            unclassified.append(item[i+1])
    done.append(grouped)
    if unclassified != []:
        unclassified_group.append(unclassified)

In [989]:
# Testing
lst = list(itertools.chain(*done))
#print(len(lst))
#print(len(unique))
clst = list(itertools.chain(*unclassified_group))
#print(len(clst))
#unclassified_group

In [990]:
for item in unclassified_group:
    if len(item) != 1:
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        #print(temp_unclassified)
        if temp_unclassified != [] and len(temp_unclassified) == 1:
            unique.append(temp_unclassified[0])
        elif temp_unclassified != [] and len(temp_unclassified) > 1:
            unique.append(temp_unclassified[:len(temp_unclassified)])
    else:
        unique.append(item[0])
#unique 

In [991]:
# Testing
lst = list(itertools.chain(*done))
#print(len(lst))
#print(len(unique))
#unique

In [992]:
### DICTIONARY OF STORE NAME
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]
#dict_done

In [993]:
### FLATTEN STORE NAME LIST
all_name_lst = list(itertools.chain(*done))
all_name_lst = all_name_lst + unique
len(all_name_lst)

889

In [994]:
### ANNOTATION_MAP
map_store_id = []
map_idx = []

for m in list(map_['stem_map_store']):
    for key, v in dict_done.items():
        if m in v:
            position = v[v.index(m)]
            map_store_id.append(all_name_lst.index(position)+1)
            map_idx.append(position)
            break
        elif m in unique:
            map_store_id.append(all_name_lst.index(m)+1)
            break
        elif (m not in unique) and (key == list(dict_done.keys())[-1]):
            map_store_id.append('9999')
  
#map_idx 

In [995]:
print(len(set(map_idx)))
print(len(map_idx))

230
230


In [996]:
### ANNOTATION_FACEBOOK
fb_store_id = []

for f in list(fb_['stem_store']):
    for key, v in dict_done.items():
        repeat = list(set(v) & set(map_idx))
        if (f in v) and (repeat != []): 
            fb_store_id.append(all_name_lst.index(repeat[0])+1)   
            break
        elif (f in v) and (repeat == []):
            fb_store_id.append(all_name_lst.index(v[0])+1)
            break
        elif f in unique:
            fb_store_id.append(all_name_lst.index(f)+1)
            break
        elif (f not in unique) and (key == list(dict_done.keys())[-1]):
            fb_store_id.append('9999')


In [997]:
len(fb_store_id)

1258

In [998]:
### ANNOTATION_FACEBOOK
'''fb_store_id = []

for store in list(fb_['stem_store']):
    for key, v in dict_done.items():
        if store in v:
            fb_store_id.append(key)
            break
        elif store in unique:
            fb_store_id.append(unique.index(store)+500)
            break
        elif (store not in unique) and (key == list(dict_done.keys())[-1]):
            fb_store_id.append('9999')'''

"fb_store_id = []\n\nfor store in list(fb_['stem_store']):\n    for key, v in dict_done.items():\n        if store in v:\n            fb_store_id.append(key)\n            break\n        elif store in unique:\n            fb_store_id.append(unique.index(store)+500)\n            break\n        elif (store not in unique) and (key == list(dict_done.keys())[-1]):\n            fb_store_id.append('9999')"

In [999]:
### ANNOTATION_MAP
'''map_store_id = []

for store in list(map_['stem_map_store']):
    for key, v in dict_done.items():
        if store in v:
            map_store_id.append(key)
            break
        elif store in unique:
            map_store_id.append(unique.index(store)+500)
            break
        elif (store not in unique) and (key == list(dict_done.keys())[-1]):
            map_store_id.append('9999')'''

"map_store_id = []\n\nfor store in list(map_['stem_map_store']):\n    for key, v in dict_done.items():\n        if store in v:\n            map_store_id.append(key)\n            break\n        elif store in unique:\n            map_store_id.append(unique.index(store)+500)\n            break\n        elif (store not in unique) and (key == list(dict_done.keys())[-1]):\n            map_store_id.append('9999')"

In [1000]:
### Create post_id 
post_id = [i for i in range(len(fb_store_id))]
random.shuffle(post_id)

In [1001]:
### OUTPUT CSV
fb_['post_id'] = post_id
fb_['store_id'] = fb_store_id
map_['store_id'] = map_store_id
fb_ = fb_.drop(columns=['stem_store'])
map_ = map_.drop(columns=['stem_map_store'])
fb_ = fb_[['post_id', 'stores','store_id', 'create_on', 'ramen_name','fb_review']]
map_ = map_[['store_id','store', 'address', 'discription', 'openTime', 'latitude', 'longtitute',
       'mapReview', 'region', 'soup', 'transport']]
fb_.to_csv('fb_store_id_done.csv', index=False)
map_.to_csv('map_store_id_done.csv', index=False)

In [1002]:
#map_.columns

In [1003]:
len(set(fb_store_id))

378